# Model Inference

Here, I show how the model can be used. This works if the model is deployed
using flask as well as docker.
Just choose one of both.
To make this work, you first have to start the application.
Make sure your current working
directory is the base directory of this repository: `RestAI/`.
Now run one of the two provided commands.
Then you can query the model using the
code in this notebook.

## Flask

```bash
# start application
python scripts/predict.py
```

## Docker

```bash
# build the image
docker build -t restai .

# Run the container
# -p maps host port to container port (host:container)
docker run -p 9696:9696 restai
# if that doesn't work (may happen if you use bash instead of zsh, run:)
# docker run -p 9696::9696 restai
```

## Preparations

In [3]:
# dependencies
import requests
import pandas as pd
from flask import jsonify

# API endpoints
url_transformed = "http://localhost:9696/predict_transformed"
url_untransformed = "http://localhost:9696/predict_untransformed"

## Transformed Data
Here, transformed means encoded and scaled already.

In [4]:
# load the test data
X_test = pd.read_csv("../data/processed/main/sleep_data_main_test_features.csv")

# get one test case from test data
testcase = pd.DataFrame(X_test.iloc[64]).T
testcase

# prepare test case (convert Series to plain dictionary)
testcase_dict = testcase.to_dict(orient="records")[0]

testcase_dict

{'age': 1.0,
 'bedtime': 0.4519929453262786,
 'caffeine_intake': 0.8,
 'gender=female': 1.0,
 'gender=male': 0.0,
 'gender=other': 0.0,
 'physical_activity': 0.075,
 'screen_time': 0.4000000000000001,
 'study_hours': 0.773109243697479,
 'university_year=1st_Year': 0.0,
 'university_year=2nd_Year': 0.0,
 'university_year=3rd_Year': 1.0,
 'university_year=4th_Year': 0.0,
 'wakeup_time': 0.7607505863956217}

In [5]:
# query the model and print response
response = requests.post(url_transformed, json=testcase_dict)
print(response.json())

{'sleep_quality': [0.5280619344669989]}


## Untransformed Data
Here, untransformed means not yet encoded or scaled.
For this, one hot encoding and scaling yet needs to be done.
This is a more realistic use case.

In [4]:
# construct an example
testcase_untransformed = {
    # columns in need of normalization
        # problem: must be normalized same way as train data
        "age": 19,
        "bedtime": 0.5,
        "caffeine_intake": 1,
        "gender": "male",
        "physical_activity": 1,
        "screen_time": 1,
        "study_hours": 10,
        "university_year": "2nd_year",
        "wakeup_time": 1
    }

# display testcase
testcase_untransformed

{'age': 19,
 'bedtime': 0.5,
 'caffeine_intake': 1,
 'gender': 'male',
 'physical_activity': 1,
 'screen_time': 1,
 'study_hours': 10,
 'university_year': '2nd_year',
 'wakeup_time': 1}

In [5]:
# query the model
# one hot encoding and scaling are handled inside the predict function
response = requests.post(url_untransformed, json=testcase_untransformed)
print(response.json())

{'sleep_quality': [0.4655126149946922]}
